# Overture Divisions: Israel to GeoParquet

This notebook reads the Overture Maps `divisions` theme for Israel (`country = 'IL'`) from the public S3 release and saves the results locally as a GeoParquet file in `data/`.

References:
- Overture Maps Divisions: https://docs.overturemaps.org/guides/divisions/
- Pattern inspired by `notebooks/duckdb_demo.ipynb`


In [1]:
import os
import duckdb
import pandas as pd
import geopandas as gpd

# Configure release and output paths
OVERTURE_REGION = 'us-west-2'
OVERTURE_RELEASE = '2025-07-23.0'  # match the demo; adjust if needed
DIVISIONS_PATH = f"s3://overturemaps-{OVERTURE_REGION}/release/{OVERTURE_RELEASE}/theme=divisions/type=division/*"
OUTPUT_DIR = 'data'
OUTPUT_FILE = os.path.join(OUTPUT_DIR, 'overture_divisions_israel.parquet')  # GeoParquet with metadata

os.makedirs(OUTPUT_DIR, exist_ok=True)

con = duckdb.connect()
con.execute("INSTALL httpfs; LOAD httpfs;")
con.execute("INSTALL spatial; LOAD spatial;")
con.execute("SET s3_region=?;", [OVERTURE_REGION])
con.execute("SET python_scan_all_frames=true;")


In [2]:
# Peek at counts for Israel to confirm availability
preview_counts = con.execute(
    f"""
    SELECT country, COUNT(*) AS cnt
    FROM read_parquet('{DIVISIONS_PATH}', hive_partitioning=1)
    WHERE country = 'IL'
    GROUP BY country
    """
).df()
preview_counts

,country,cnt
0,IL,2723


In [3]:
# Read divisions for Israel.
# Use ST_AsWKB on geometry so we can build a GeoDataFrame reliably.
df_il = con.execute(
    f"""
    SELECT
      id,
      country,
      subtype,
      class,
      names,
      region,
      local_type,
      parent_division_id,
      population,
      wikidata,
      ST_AsWKB(geometry) AS geom_wkb
    FROM read_parquet('{DIVISIONS_PATH}', hive_partitioning=1)
    WHERE country = 'IL'
    """
).df()
df_il.head()

,id,country,subtype,class,names,region,local_type,parent_division_id,population,wikidata,geom_wkb
0,e2b8eef7-a2b7-41e7-b174-b7fefef74e6b,IL,neighborhood,None,"{'primary': '6 שחמון', 'common': {'en': 'Shaha...",IL-D,{'en': 'neighborhood'},aeff3c01-fd80-4365-82ea-37d0876252ec,NaN,None,"[1, 1, 0, 0, 0, 66, 75, 128, 132, 166, 119, 65..."
1,b8d6855a-f913-456c-b4b9-76c2fe8e43ee,IL,macrohood,None,"{'primary': 'שחמון', 'common': {'hr': 'Šahamon...",IL-D,{'en': 'suburb'},aeff3c01-fd80-4365-82ea-37d0876252ec,NaN,Q124399955,"[1, 1, 0, 0, 0, 23, 119, 205, 245, 199, 119, 6..."
2,bb16f5ca-1eec-40c9-9f96-4438bdc5852a,IL,neighborhood,None,"{'primary': 'מצפה ים', 'common': {'en': 'Mitsp...",IL-D,{'en': 'neighbourhood'},aeff3c01-fd80-4365-82ea-37d0876252ec,NaN,None,"[1, 1, 0, 0, 0, 35, 240, 44, 230, 140, 119, 65..."
3,39c7c3d3-001c-4113-9607-a08664e80515,IL,neighborhood,None,"{'primary': 'צאלים', 'common': {'en': 'Tseelim...",IL-D,{'en': 'neighbourhood'},aeff3c01-fd80-4365-82ea-37d0876252ec,NaN,None,"[1, 1, 0, 0, 0, 240, 98, 171, 21, 183, 119, 65..."
4,d44807a1-59d9-421e-ab32-8a68254c7641,IL,neighborhood,None,"{'primary': 'גנים א', 'common': {'en': 'Ganim ...",IL-D,{'en': 'neighbourhood'},aeff3c01-fd80-4365-82ea-37d0876252ec,NaN,None,"[1, 1, 0, 0, 0, 89, 60, 171, 187, 104, 120, 65..."


In [4]:
# Build GeoDataFrame and save as GeoParquet
gdf_il = gpd.GeoDataFrame(
    df_il.drop(columns=['geom_wkb']),
    geometry=gpd.GeoSeries.from_wkb(df_il['geom_wkb'], crs='EPSG:4326')
)

# Save with GeoParquet metadata
gdf_il.to_parquet(OUTPUT_FILE, index=False)

OUTPUT_FILE, len(gdf_il), gdf_il.geometry.iloc[0].geom_type

TypeError: Expected bytes or string, got bytearray

In [ ]:
# Optional: quick visual check (may be heavy to render all)
ax = gdf_il.sample(min(1000, len(gdf_il))).plot(figsize=(6,6), alpha=0.5, edgecolor='k')
ax.set_title('Sample of Overture Divisions in Israel')
ax.set_axis_off()
ax.figure.tight_layout()
ax.figure